 # text classification using keras implement

In [42]:
from keras.models import Sequential
from keras.preprocessing import sequence
from keras.layers import Dense, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Activation, Dropout, Embedding, GlobalAveragePooling1D
from keras.datasets import imdb
from keras.layers import Merge


In [31]:
max_features = 10000
maxlen = 400
batch_size = 32
embedding_dims = 200
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 5
keep_dropout_prob=0.5

In [32]:
print('loadding data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_text), 'test sequences')

loadding data...
25000 train sequences
25000 test sequences


In [29]:
len(x_train[2])

141

In [33]:
print('Padding data (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train_shape: ', x_train.shape)
print('x_test_shape: ', x_test.shape)

Padding data (samples x time)
x_train_shape:  (25000, 400)
x_test_shape:  (25000, 400)


## build model 1

In [38]:
print('build model')
model = Sequential()

# embedding layer
model.add(Embedding(max_features, embedding_dims, input_length=maxlen))
model.add(Dropout(keep_dropout_prob))

# convolution layer
model.add(Conv1D(filters,
                [kernel_size],
                padding='valid',
                activation='relu',
                strides=1))

# max pooling
model.add(GlobalMaxPooling1D())

# FC layer
model.add(Dense(hidden_dims))
model.add(Dropout(keep_dropout_prob))
model.add(Activation('relu'))

# sigmoid layer
model.add(Dense(1))
model.add(Activation('sigmoid'))

build model


## build model 2

In [39]:
print('build model')
model = Sequential()

# embedding layer
model.add(Embedding(max_features, embedding_dims, input_length=maxlen))
model.add(Dropout(keep_dropout_prob))

# convolution layer
model.add(Conv1D(filters,
                kernel_size,
                padding='valid',
                activation='relu',
                strides=1))
model.add(Conv1D(filters,
                kernel_size,
                padding='valid',
                activation='relu',
                strides=1))

model.add(MaxPooling1D(kernel_size))

model.add(Conv1D(filters *2, kernel_size, activation='relu'))
model.add(Conv1D(filters *2, kernel_size, activation='relu'))
# max pooling
model.add(GlobalMaxPooling1D())

# FC layer
#model.add(Dense(hidden_dims))
#model.add(Dropout(keep_dropout_prob))
#model.add(Activation('relu'))

# sigmoid layer
model.add(Dense(1))
model.add(Activation('sigmoid'))

build model


## build model 3 using different filter_size

In [46]:
print('using different filter size to build CNN model for text classification')
filter_size = [3, 4, 5]
model = Sequential()
convs = []
for i in filter_size:
    subconv = Sequential()
    subconv.add(Embedding(max_features, embedding_dims, input_length=maxlen))
    subconv.add(Conv1D(filters,
                      i,
                    padding='valid'))
    subconv.add(GlobalMaxPooling1D())
    convs.append(subconv)

model.add(Merge(convs, mode='concat'))

#FC 
model.add(Dense(hidden_dims))
model.add(Dropout(keep_dropout_prob))
model.add(Activation('relu'))

#output layer
model.add(Dense(1))
model.add(Activation('sigmoid'))

## special train and prediction
# train and evalution
model.compile(loss='binary_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

model.fit([x_train, x_train, x_train], 
          y_train, epochs=epochs, batch_size=batch_size, 
          validation_data=([x_test, x_test, x_test], y_test))
# model.evalute


using different filter size to build CNN model for text classification


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:14: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  


Train on 25000 samples, validate on 25000 samples
Epoch 1/5
25000/25000 [==============================] - 429s - loss: 0.3643 - acc: 0.8281 - val_loss: 0.2484 - val_acc: 0.8993
Epoch 2/5
25000/25000 [==============================] - 428s - loss: 0.1427 - acc: 0.9494 - val_loss: 0.2478 - val_acc: 0.9001
Epoch 3/5
25000/25000 [==============================] - 428s - loss: 0.0374 - acc: 0.9884 - val_loss: 0.3397 - val_acc: 0.8963
Epoch 4/5
25000/25000 [==============================] - 429s - loss: 0.0167 - acc: 0.9942 - val_loss: 0.4326 - val_acc: 0.8952
Epoch 5/5
25000/25000 [==============================] - 429s - loss: 0.0171 - acc: 0.9940 - val_loss: 0.5454 - val_acc: 0.8760


In [35]:
# train and evalution
model.compile(loss='binary_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(x_test, y_test))
# model.evalute

ValueError: Error when checking target: expected dropout_9 to have 3 dimensions, but got array with shape (25000, 1)